# A small League of Legend items analysis

Code by Igor Grégoire assisted by copilot

### Importing packages

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
import plotly.express as px
from sklearn.metrics import silhouette_samples, silhouette_score

## Preprocessing the dataset

| Feature               | Description                                                                                      |
|-----------------------|--------------------------------------------------------------------------------------------------|
| Item                  | Full name of the item                                                                            |
| Cost                  | Cost of the item                                                                                 |
| Sell                  | Sell of the item                                                                                 |
| AD                    | Attack Damage                                                                                    |
| AS                    | Attack Speed                                                                                     |
| Crit                  | Chance of critical hit                                                                           |
| LS                    | Life steal                                                                                       |
| APen                  | Ability Penetration                                                                              |
| AP                    | Ability Power                                                                                    |
| AH                    | Ability Haste                                                                                    |
| Mana                  | Mana                                                                                             |
| MP5                   | Mana Regeneration per 5s                                                                         |
| HSP                   | Heal/Shield Power                                                                                |
| OVamp                 | Omnivamp                                                                                         |
| MPen                  | Magic Penetration                                                                                |
| Health                | Health                                                                                           |
| Armor                 | Armor                                                                                            |
| MR                    | Magic Resistance                                                                                 |
| HP5                   | Health regeneration per 5s                                                                       |
| MS                    | Movement Speed                                                                                   |
| Maps                  | Maps where the item is available                                                                 |

### Getting the dataset

In [16]:
df = pd.read_csv('LOL_items_stats.csv', sep=';')
df.head()

,Item,Cost,Sell,AD,AS,Crit,LS,APen,AP,AH,...,MP5,HSP,OVamp,MPen,Health,Armor,MR,HP5,MS,Maps
0,Abyssal Mask,2700,1890.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,400.0,NaN,60.0,NaN,NaN,All
1,Aegis of the Legion,1500,1050.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,30.0,30.0,NaN,NaN,All
2,Aether Wisp,850,595.0,NaN,NaN,NaN,NaN,NaN,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All
3,Amplifying Tome,435,304.5,NaN,NaN,NaN,NaN,NaN,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All
4,Archangel's Staff,3000,2100.0,NaN,NaN,NaN,NaN,NaN,65.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All


### Dataset curation

In [17]:
columns_to_keep = ["Cost", 
                   "Sell", 
                   "AD", 
                   "AS", 
                   "Crit", 
                   "LS", 
                   "APen", 
                   "AP", 
                   "AH", 
                   "Mana", 
                   "MP5", 
                   "HSP", 
                   "OVamp", 
                   "MPen", 
                   "Health", 
                   "Armor",
                   "MR",
                   "HP5",
                   "MS"]

# List of items names
names = df["Item"]
df_keep = df[columns_to_keep]


### Removing the NaN values

In [18]:
# Replace NaN values with 0
pd.options.mode.chained_assignment = None
df_keep.fillna(0, inplace=True)

### Scaling the dataset

In [19]:
def scale_data(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

X = scale_data(df_keep)

## Clustering

### Fitting the model

In [20]:
def kmeans_clustering(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=69)
    kmeans.fit(data)
    return kmeans.labels_

def spectral_clustering(data, n_clusters):
    spectral_clustering = SpectralClustering(n_clusters=n_clusters, random_state=69)
    spectral_clustering.fit(data)
    return spectral_clustering.labels_

### Chosing the number of cluster

In [31]:
# Not really useful...
def find_best_n_clusters(data, max_clusters):
    silhouette_scores = []
    for n_clusters in range(2, max_clusters+1):
        z = kmeans_clustering(data, n_clusters)
        silhouette_avg = silhouette_score(data, z)
        silhouette_scores.append(silhouette_avg)
    return silhouette_scores

silhouette_scores = find_best_n_clusters(X, 100)
px.line(x=range(2, 101), y=silhouette_scores, title="Silhouette score for different number of clusters")



### Fixing the clustering

In [22]:
n_clusters = 3
y = kmeans_clustering(X, n_clusters=n_clusters)
#y = spectral_clustering(X, n_clusters=n_clusters)

## Visualization

In [23]:
def normalize_data(data):
    return (data - data.min()) / (data.max() - data.min())

### 2D PCA

In [24]:
pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X)

X_pca2 = pd.DataFrame(X_pca2, columns=["PC1", "PC2"])

# Normalize the data
X_pca2["PC1"] = normalize_data(X_pca2["PC1"])
X_pca2["PC2"] = normalize_data(X_pca2["PC2"])

In [25]:
fig = px.scatter(X_pca2, x='PC1', y='PC2', color=y, color_continuous_scale="Portland", hover_name=names,width=800, height=800)
fig.update_layout(title="Items Visualization in 3D")
fig.show()

### 3D PCA

In [26]:
pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X)

X_pca3 = pd.DataFrame(X_pca3, columns=["PC1", "PC2", "PC3"])

# Normalize the data
X_pca3["PC1"] = normalize_data(X_pca3["PC1"])
X_pca3["PC2"] = normalize_data(X_pca3["PC2"])
X_pca3["PC3"] = normalize_data(X_pca3["PC3"])


In [27]:
# Plot the data in 3D
fig = px.scatter_3d(X_pca3, x='PC1', y='PC2', z='PC3', color=y, color_continuous_scale="Portland", hover_name=names)
fig.update_layout(title="Items Visualization in 3D")
fig.update_traces(marker=dict(size=3))
fig.show()


### Which items are similar ?

In [28]:
df_clusters = pd.DataFrame({"Item": names, "Cluster": y})

for i in range(n_clusters):
    print("#"*10 + " Cluster", i + 1, "#"*10)
    for item in df_clusters[df_clusters["Cluster"] == i]["Item"]:
        print(item)
    print("\n")

########## Cluster 1 ##########
Aegis of the Legion
Aether Wisp
Amplifying Tome
B. F. Sword
Bami's Cinder
Bandleglass Mirror
Berserker's Greaves
Black Mist Scythe
Blasting Wand
Blighting Jewel
Boots
Boots of Swiftness
Bramble Vest
Bulwark of the Mountain
Caulfield's Warhammer
Chain Vest
Cloak of Agility
Cloth Armor
Crystalline Bracer
Cull
Dagger
Dark Seal
Doran's Blade
Doran's Ring
Doran's Shield
Executioner's Calling
Faerie Charm
Fiendish Codex
Forbidden Idol
Frostfang
Giant's Belt
Glacial Buckler
Guardian's Blade
Guardian's Hammer
Guardian's Horn
Guardian's Orb
Harrowing Crescent
Hearthbound Axe
Hexdrinker
Hextech Alternator
Ionian Boots of Lucidity
Ironspike Whip
Kindlegem
Kircheis Shard
Last Whisper
Leeching Leer
Long Sword
Lost Chapter
Mejai's Soulstealer
Mercury's Treads
Needlessly Large Rod
Negatron Cloak
Noonquiver
Null-Magic Mantle
Oblivion Orb
Pauldrons of Whiterock
Phage
Pickaxe
Plated Steelcaps
Quicksilver Sash
Rageknife
Recurve Bow
Rejuvenation Bead
Relic Shield
Ruby Cryst